In [2]:
import pandas as pd

In [3]:
# A dictionary used for loading the dataset files into dataframes
DATAFRAMES = {}

In [4]:
## RUNS ON GOOGLE COLABS
## ONLY REQUIRED TO LOAD FILES FROM LOCAL HARD DRIVE

import io
from google.colab import files

def colab_import_local_data():
  """
  Populates DATAFRAMES dictionary with dataframes created from uploaded CSV
  files. The dictionary can be indexed using the filename.

  NOTE :: Existing content will be overwritten if the filename is the same
  as that of a previously imported file.
  """
  uploaded = files.upload()
  for k in uploaded.keys():
    DATAFRAMES[k] = pd.read_csv(io.BytesIO(uploaded[k]), header=None)

colab_import_local_data()

Saving Games.csv to Games.csv
Saving Teams.csv to Teams.csv


In [5]:
DATAFRAMES.keys()

dict_keys(['Games.csv', 'Teams.csv'])

In [6]:
for f in DATAFRAMES:
  print(f)
  print(DATAFRAMES[f].columns)

#DATAFRAMES["Games.csv"].columns

Games.csv
Int64Index([0, 1, 2, 3, 4], dtype='int64')
Teams.csv
Int64Index([0, 1], dtype='int64')


In [7]:
DATAFRAMES["Games.csv"].columns = ["GameDate", "HomeTeam", "HomeScore", "AwayTeam", "AwayScore"]
DATAFRAMES["Teams.csv"].columns = ["Conference", "Team"]

print(DATAFRAMES["Games.csv"].head(5))

print(DATAFRAMES["Teams.csv"].head(5))

     GameDate       HomeTeam  HomeScore     AwayTeam  AwayScore
0  2015-11-13         Hawaii         87  Montana St.         76
1  2015-11-13  Eastern Mich.         70      Vermont         50
2  2015-11-13       Columbia        107         Kean         62
3  2015-11-13     La.-Monroe         88      McMurry         43
4  2015-11-13           Yale         70    Fairfield         57
     Conference         Team
0  America East      Vermont
1  America East  Stony Brook
2  America East         UMBC
3  America East     Hartford
4  America East       Albany


In [8]:
df_t = DATAFRAMES["Teams.csv"]
df_g = DATAFRAMES["Games.csv"]

print("Conference Teams: {}; Home Teams: {}; Away Teams: {}".format(df_t.Team.nunique(), df_g.HomeTeam.nunique(), df_g.AwayTeam.nunique()))

Conference Teams: 353; Home Teams: 452; Away Teams: 1022


Clearly there is some clean up work to be done here. There are 353 unique team names in the Teams dataframe and this should be reflected in the Games dataframe.

In [9]:
teams = set(df_t.Team.unique())
h_teams = set(df_g.HomeTeam.unique())
a_teams = set(df_g.AwayTeam.unique())

In [10]:
print(h_teams - teams)

{'Cal St. Fullerton', 'Loyola Maryland', 'Southern U.', 'William &amp; Mary', 'St. Francis (NY)', 'UT Arlington ', 'Southeast Mo. St.', 'Savannah St.', 'UC Santa Barb.', 'Southeastern La.', 'Eastern Ill.', 'Eastern Mich.', 'Bryant', 'St. Francis (B&#039;klyn)', 'Chicago St.', 'SIU Edwardsville', 'St. John&#039;s (NY)', 'North Dakota St.', 'South Carolina St.', 'East Tenn. St.', 'Northern Ky.', 'Fresno St.', 'S. Carolina St.', 'Western Caro.', 'UNCG', 'Ave Maria', 'McNeese ', 'Jacksonville St.', 'Army West Point', 'Omaha', 'Indiana St.', 'Prairie View', 'Murray St.', 'McNeese St.', 'VCU', 'Mt. St. Mary&#039;s', 'Mid-Atlantic Christ.', 'Cleveland St.', 'Appalachian St.', 'Cal St. Northridge', 'Miami (Ohio)', 'Alabama A&amp;M', 'Penn St.', 'IPFW', 'Kansas St.', 'Coastal Caro.', 'Mississippi St.', 'N&#039;western St.', 'Coppin St.', 'St. Mary&#039;s (Cal.)', 'Mississippi Val.', 'Michigan St.', 'Tennessee St.', 'FGCU', 'Alabama St.', 'N.C. Central', 'A&amp;M-Corpus Chris', 'Southern Univ.',

In [11]:
def replace_entities(s):
  if isinstance(s, str):
    return s.replace("&amp;", "&").replace("&#039;", "'")
  else:
    return ""

df_g["HTeam"] = df_g["HomeTeam"].apply(replace_entities)
df_g["ATeam"] = df_g["AwayTeam"].apply(replace_entities)


In [12]:
import re

EXPANSIONS = [("Ala.", "Alabama"),
              ("Alas.", "Alaska"),
              ("Ariz.", "Arizona"),
              ("Ark.", "Arkansas"),
              ("Barb.", "Barbara"),
              ("(B'klyn)", "(BKN)"),
              ("(Cal.)", "(CA)"),
              ("Cal St.", "Cal State"),
              ("Caro.", "Carolina"),
              ("C.C.", "CC"),
              ("Corpus Chris", "CC"),
              ("Cent.", "Central"),
              ("Colo.", "Colorado"),
              ("Conn.", "Connecticut"),
              ("CSU", "Cal State"),
              ("D'son", "Dickinson"),
              ("East.", "Eastern"),
              ("(Fla.)", "(FL)"),
              ("Fla.", "Florida"),
              ("Ga.", "Georgia"),
              ("Geo.", "George"),
              ("Ill.", "Illinois"),
              ("La.", "Louisiana"),
              ("Ky.", "Kentucky"),
              ("Mass.", "Massachusetts"),
              ("(Md.)", "(MD)"),
              ("Md.", "Maryland"),
              ("Mich.", "Michigan"),
              ("Miss.", "Mississippi"),
              ("Mo.", "Missouri"),
              ("Mt.", "Mount"),
              ("N.C.", "North Carolina"),
              ("Neb.", "Nebraska"),
              ("N'western", "Northwestern"),
              ("(Ohio)", "(OH)"),
              ("(Pa.)", "(PA)"),
              ("S.C.", "South Carolina"),
              ("S.", "South"),
              ("So.", "Southern"),
              ("Tenn.", "Tennessee"),
              ("U.", "University"),
              ("Univ.", "University"),
              ("Val.", "Valley"),
              ("Wash.", "Washington")
              ]

SET_NEEDS_STATE = set(["Grambling",
                       "Long Beach",
                       "McNeese",
                       "Mississippi",
                       "Mississippi Valley"])



FULL_REPLACEMENTS = [("A&M-CC", "Texas A&M-CC"),
                     ("Albany (NY)", "Albany"),
                     ("American", "American University"),
                     ("Army West Point", "Army"),
                     ("Bakersfield", "Cal State Bakersfield"),
                     ("Bryant", "Bryant University"),
                     ("Cal Baptist", "California Baptist"),
                     ("Charleston South", "Charleston Southern"),
                     ("Col. of Charleston", "Charleston"),
                     ("Detroit", "Detroit Mercy"),
                     ("FGCU", "Florida Gulf Coast"),
                     ("FIU", "Florida International"),
                     ("Fort Wayne", "Purdue Fort Wayne"),
                     ("IPFW", "Purdue Fort Wayne"),
                     ("Lamar University", "Lamar"),
                     ("Little Rock", "Arkansas-Little Rock"),
                     ("LMU", "Loyola Marymount"),
                     ("Long Island", "LIU Brooklyn"),
                     ("Louisiana", "Louisiana-Lafayette"),
                     ("Loyola Chicago", "Loyola (IL)"),
                     ("Loyola Maryland", "Loyola (MD)"),
                     ("LSU Shreveport", "LSU"),
                     ("NC State", "North Carolina State"),
                     ("Omaha", "Nebraska-Omaha"),
                     ("Penn", "Pennsylvania"),
                     ("Prairie View", "Prairie View A&M"),
                     ("SIUE", "SIU-Edwardsville"),
                     ("UALR", "Arkansas-Little Rock"),
                     ("UConn", "Connecticut"),
                     ("UIC", "Illinois-Chicago"),
                     ("UMES", "Maryland-Eastern Shore"),
                     ("UNCG", "UNC Greensboro"),
                     ("UNC Wilmington", "North Carolina-Wilmington"),
                     ("UNCW", "North Carolina-Wilmington"),
                     ("UNI", "Northern Iowa"),
                     ("UT Arlington", "Texas-Arlington"),
                     ("UT Martin", "Tennessee-Martin"),
                     ("UTRGV", "Texas Rio Grande Valley"),
                     ("Saint Joseph's", "Saint Joseph's (PA)"),
                     ("Seattle U", "Seattle"),
                     ("SFA", "Stephen F Austin"),
                     ("Southern California", "USC"),
                     ("South Carolina Upstate", "USC Upstate"),
                     ("Southern Mississippi", "Southern Miss"),
                     ("St Francis (NY)", "St Francis (BKN)"),
                     ("St John's (NY)", "St John's"),
                     ("St Mary's (CA)", "Saint Mary's"),
                     ("VCU", "Virginia Commonwealth"),
                     ("VMI", "Virginia Military")]


def team_names_cleanup(s):
  if isinstance(s, str):
    # Remove trailing white space
    s = s.strip()
    # Expand abbreviations
    for a,b in EXPANSIONS:
      s = s.replace(a, b)
    # Expand 'St.' cases
    q = re.compile("\s+St\.$", re.IGNORECASE)
    s = re.sub(q, " State", s)
    # Handle all other 'St.' cases
    p = re.compile("St\.", re.IGNORECASE)
    s = re.sub(p, "St", s)
    # 
    if s in SET_NEEDS_STATE:
      s += " State"
    # Catch all
    for a,b in FULL_REPLACEMENTS:
      if s == a:
        s = b

  return s

def expand_state(s):
  q = re.compile("\s+St\.$", re.IGNORECASE)
  s = re.sub(q, " State", s)
  return s

#    p = re.compile("St\.", re.IGNORECASE)
#    s = re.sub(p, "St", s)


def append_state_if_needed(s):
  if isinstance(s, str):
    for a in NEEDS_STATE:
      p = re.compile(a+"$")
      m = p.match(s)
      if m:
        s += " State"
  return s

#df_g["HTeam"] = df_g["HTeam"].apply(expand_abbreviations)
#df_g["ATeam"] = df_g["ATeam"].apply(expand_abbreviations)

#df_g["HTeam"] = df_g["HTeam"].apply(expand_state)
#df_g["ATeam"] = df_g["ATeam"].apply(expand_state)

df_g["HTeam"] = df_g["HTeam"].apply(team_names_cleanup)
df_g["ATeam"] = df_g["ATeam"].apply(team_names_cleanup)


In [13]:
print(df_g.HTeam.nunique(), df_g.ATeam.nunique())

365 924


In [14]:
set(df_t.Team) - set(df_g.HTeam.unique())

set()

In [15]:
set(df_g.HTeam.unique()) - set(df_t.Team)

{'Alaska Anchorage',
 'Ave Maria',
 'Cal StateN',
 'Centenary (LA)',
 'Chaminade',
 'Humboldt State',
 'Mid-Atlantic ChriSt',
 'Nebraska Omaha',
 'Rowan',
 'SIU Edwardsville',
 'Stephen F. Austin',
 'University of the Southwest'}

In [16]:
def transform_team_name(s):
  unwanted = "!#$%&'()*+,./:;<=>?@[\]^_`{|}~"
  return (s.translate(str.maketrans("", "", unwanted))
           .lower()
           .replace(" ", "-")
           .strip())


df_g["HTeam_"] = df_g["HTeam"].apply(transform_team_name)
df_g["ATeam_"] = df_g["ATeam"].apply(transform_team_name)
df_t["Team_"] = df_t["Team"].apply(transform_team_name)

In [17]:
print(df_g.head())

     GameDate       HomeTeam  ...            HTeam_         ATeam_
0  2015-11-13         Hawaii  ...            hawaii  montana-state
1  2015-11-13  Eastern Mich.  ...  eastern-michigan        vermont
2  2015-11-13       Columbia  ...          columbia           kean
3  2015-11-13     La.-Monroe  ...  louisiana-monroe        mcmurry
4  2015-11-13           Yale  ...              yale      fairfield

[5 rows x 9 columns]


In [18]:
df_games = df_g.copy()

In [20]:
df_games.shape

(23493, 9)

In [21]:
merged = pd.merge(df_games, df_t, left_on=['HTeam_'], right_on=['Team_'])
merged.shape

(23454, 12)

In [22]:
merged.head()

,GameDate,HomeTeam,HomeScore,AwayTeam,AwayScore,HTeam,ATeam,HTeam_,ATeam_,Conference,Team,Team_
0,2015-11-13,Hawaii,87,Montana St.,76,Hawaii,Montana State,hawaii,montana-state,Big West,Hawaii,hawaii
1,2015-11-15,Hawaii,74,Coastal Caro.,63,Hawaii,Coastal Carolina,hawaii,coastal-carolina,Big West,Hawaii,hawaii
2,2015-11-17,Hawaii,76,Nevada,75,Hawaii,Nevada,hawaii,nevada,Big West,Hawaii,hawaii
3,2015-11-22,Hawaii,99,Nicholls St.,74,Hawaii,Nicholls State,hawaii,nicholls-state,Big West,Hawaii,hawaii
4,2015-12-02,Hawaii,75,Ark.-Pine Bluff,47,Hawaii,Arkansas-Pine Bluff,hawaii,arkansas-pine-bluff,Big West,Hawaii,hawaii


In [23]:
merged2 = pd.merge(merged, df_t, left_on=['ATeam_'], right_on=['Team_'])
merged2.shape

(21783, 15)

In [24]:
merged2.head()

,GameDate,HomeTeam,HomeScore,AwayTeam,AwayScore,HTeam,ATeam,HTeam_,ATeam_,Conference_x,Team_x,Team__x,Conference_y,Team_y,Team__y
0,2015-11-13,Hawaii,87,Montana St.,76,Hawaii,Montana State,hawaii,montana-state,Big West,Hawaii,hawaii,Big Sky,Montana State,montana-state
1,2016-01-14,North Dakota,85,Montana St.,68,North Dakota,Montana State,north-dakota,montana-state,Summit League,North Dakota,north-dakota,Big Sky,Montana State,montana-state
2,2018-02-01,North Dakota,75,Montana St.,74,North Dakota,Montana State,north-dakota,montana-state,Summit League,North Dakota,north-dakota,Big Sky,Montana State,montana-state
3,2018-03-06,North Dakota,0,Montana St.,0,North Dakota,Montana State,north-dakota,montana-state,Summit League,North Dakota,north-dakota,Big Sky,Montana State,montana-state
4,2017-12-05,Central Mich.,75,Montana St.,48,Central Michigan,Montana State,central-michigan,montana-state,Mid-American,Central Michigan,central-michigan,Big Sky,Montana State,montana-state


In [25]:
merged2.to_csv('cleaned-data.csv',mode = 'w', index=False)